In [1]:
## A script for practicing ML on text data to predict pet adoption speed based on description.
## Script name: predict_adoption_speed_by_description.py
## Date: 01/31/2019
## Author: Wei Guo
## Data source: Gaggle.com PetAdoptionSpeed competition

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

## Load data
train = pd.read_csv("train.csv").fillna(' ')
test  = pd.read_csv("test.csv").fillna(' ')
desc_train=train.Description
y = train['AdoptionSpeed']
## External test data 
text_extest_orig = test[['PetID','Description']]

text_train, text_test,y_train,y_test = train_test_split(desc_train, y, stratify=y, random_state=42)
print(text_train.shape)
print(text_train.head(5))
print(text_test.shape)
print(text_test.head(5))
print(y_train.shape)
print(y_train.head(5))
print(y_test.shape)
print(y_test.head(5))

vect = CountVectorizer(min_df=5, stop_words="english").fit(text_train)
X_train = vect.transform(text_train)
X_test  = vect.transform(text_test)
print(repr(X_train))
print(X_train.toarray())
print(repr(X_test))
#print(X_train.toarray())

# 5-fold cross validation
scores = cross_val_score(LogisticRegression(), X_train,y_train,cv=5)
print("Training mean cross val score: {:.2f}".format(np.mean(scores)))

#Grid search for best regularization parameter C and cross validation
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(),param_grid,cv=5)
grid.fit(X_train,y_train)
print(grid.best_score_)
print(grid.best_params_)

# Internal validation
print("CV score on X_test, y_test: {:.2f}".format(grid.score(X_test, y_test)))

# External test
text_extest = text_extest_orig.Description
X_extest = vect.transform(text_extest)
print(repr(X_extest))
pred = grid.predict(X_extest)

# Prepare submission
submission_df = pd.DataFrame(text_extest_orig['PetID'], columns=['PetID'])
submission_df['AdoptionSpeed'] = pred
submission_df.to_csv("AdoptionSpeed_ML_byDescription_013119.csv", index=False)

(11244,)
10864    This handsome fellow is looking for a loving h...
8669     5 adorable kittens and 1 mother cat for adopti...
6105     Looking for a good home for this kitten. Left ...
10506    Kindly contact us if interested in adopting th...
2932     I have a mix husky which is 6 yrs old, it neve...
Name: Description, dtype: object
(3749,)
12078    The puppy is currently taking shelter at SPCA ...
2475     The puppy fostered at my friend's house at Sg....
1259     A stray mum gave birth to this six female pupp...
8767                       Tame, active, cute and playful.
2540     Emma was found trapping in a drain in a heavy ...
Name: Description, dtype: object
(11244,)
10864    0
8669     3
6105     2
10506    4
2932     3
Name: AdoptionSpeed, dtype: int64
(3749,)
12078    3
2475     1
1259     3
8767     2
2540     4
Name: AdoptionSpeed, dtype: int64
<11244x4961 sparse matrix of type '<class 'numpy.int64'>'
	with 274275 stored elements in Compressed Sparse Row format>
[[0 0 0 ... 

C:\Users\guowc_000\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\guowc_000\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Training mean cross val score: 0.35


C:\Users\guowc_000\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\guowc_000\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\guowc_000\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.3528103877623621
{'C': 0.1}
CV score on X_test, y_test: 0.35
<3948x4961 sparse matrix of type '<class 'numpy.int64'>'
	with 98645 stored elements in Compressed Sparse Row format>
